## Bài 15: EC với bài toán Đa nhiệm (MFO)

### Cài đặt EC đa nhiệm cho 2 bài toán TSP nhỏ

In [4]:
import numpy as np
import random
random.seed(42)

In [29]:
def distance_matrix(coords):
    n = len(coords)
    dmat = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            d = np.linalg.norm(np.array(coords[i]) - np.array(coords[j]))
            dmat[i, j] = dmat[j, i] = d
    return dmat


def tour_length(tour, dmat):
    return sum(dmat[tour[i], tour[(i+1) % len(tour)]] for i in range(len(tour)))


def crossover_ox(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    child = [-1] * size
    child[a:b] = p1[a:b]
    fill = [x for x in p2 if x not in child]
    pos = 0
    for i in range(size):
        if child[i] == -1:
            child[i] = fill[pos]
            pos += 1
    return child

def mutate_swap(tour):
    a, b = random.sample(range(len(tour)), 2)
    tour[a], tour[b] = tour[b], tour[a]
    return tour


def mfea_tsp(coords_list, pop_size=30, gens=200):
    max_n = max(len(c) for c in coords_list)
    dmats = [distance_matrix(c) for c in coords_list]
    num_tasks = len(coords_list)
    population = []
    skill_factors = []
    for _ in range(pop_size):
        perm = list(range(max_n))
        random.shuffle(perm)
        population.append(perm)
        skill_factors.append(random.randint(0, num_tasks-1))
    def evaluate(ind, task):
        n = len(coords_list[task])
        sub_tour = [x for x in ind if x < n]
        return tour_length(sub_tour, dmats[task])
    for g in range(gens):
        offspring = []
        offspring_sf = []
        for _ in range(pop_size // 2):
            i, j = random.sample(range(pop_size), 2)
            p1, p2 = population[i], population[j]
            sf1, sf2 = skill_factors[i], skill_factors[j]
            c1 = crossover_ox(p1, p2)
            c2 = crossover_ox(p2, p1)
            if random.random() < 0.3:
                c1 = mutate_swap(c1)
            if random.random() < 0.3:
                c2 = mutate_swap(c2)
            offspring.extend([c1, c2])
            offspring_sf.extend(
                [random.choice([sf1, sf2]), random.choice([sf1, sf2])])
        population.extend(offspring)
        skill_factors.extend(offspring_sf)
        new_pop, new_sf = [], []
        for task in range(num_tasks):
            inds = [ind for ind, sf in zip(
                population, skill_factors) if sf == task]
            if not inds:
                continue
            inds.sort(key=lambda ind: evaluate(ind, task))
            top = inds[:pop_size // num_tasks]
            new_pop.extend(top)
            new_sf.extend([task] * len(top))
        while len(new_pop) < pop_size:
            i = random.randrange(len(population))
            new_pop.append(population[i])
            new_sf.append(skill_factors[i])
        population, skill_factors = new_pop, new_sf

        if g % 50 == 0:
            bests = [min([evaluate(ind, t) for ind, sf in zip(
                population, skill_factors) if sf == t]) for t in range(num_tasks)]
            print(f"Gen {g}: Best = {bests}")
    results = []
    for t in range(num_tasks):
        best_ind = min(population, key=lambda ind: evaluate(ind, t))
        n = len(coords_list[t])
        sub_tour = [x for x in best_ind if x < n]
        results.append((sub_tour, evaluate(best_ind, t)))
    return results

In [30]:
coords1 = [(random.random()*100, random.random()*100) for _ in range(20)]
coords2 = [(random.random()*100, random.random()*100) for _ in range(25)]

solutions = mfea_tsp([coords1, coords2])
for i, (tour, length) in enumerate(solutions):
    print(f"TSP {len(tour)}: length={length:.2f}, tour={tour}")

Gen 0: Best = [np.float64(763.3399583653276), np.float64(1283.6361922896463)]
Gen 50: Best = [np.float64(473.17634149744634), np.float64(828.0761731039281)]
Gen 100: Best = [np.float64(456.9802268872078), np.float64(729.6306731858375)]
Gen 150: Best = [np.float64(435.49313912735676), np.float64(632.2802955443847)]
TSP 20: length=432.49, tour=[14, 11, 7, 16, 13, 17, 2, 19, 4, 5, 12, 1, 8, 3, 6, 9, 18, 0, 10, 15]
TSP 25: length=591.78, tour=[7, 13, 11, 4, 24, 20, 2, 17, 19, 22, 3, 14, 9, 15, 18, 0, 8, 16, 5, 10, 1, 6, 23, 21, 12]
